[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AnFreTh/STREAM/blob/main/docs/notebooks/arabic_testing.ipynb)
[![Open On GitHub](https://img.shields.io/badge/Open-on%20GitHub-blue?logo=GitHub)](https://github.com/AnFreTh/STREAM/blob/main/docs/notebooks/arabic_testing.ipynb)

# Arabic Topic Modeling with STREAM

This notebook demonstrates STREAM's capabilities for Arabic text topic modeling, including specialized preprocessing and model adaptations for Arabic text.

In [2]:
# Import required packages
import pyarabic
from pyarabic.araby import strip_tashkeel, strip_tatweel
from pyarabic.normalize import normalize_hamza, normalize_lamalef
import unicodedata

# Download required NLTK data
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\yahya\my_projects\stream_env\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Loading Arabic Dataset

In [3]:
# First import TMDataset
from stream_topic.utils.dataset import TMDataset
from stream_topic.models import KmeansTM
from stream_topic.preprocessor._preprocessor import TextPreprocessor
from stream_topic.preprocessor.arabic_preprocessing import ArabicPreprocessor
from stream_topic.models.ArabicKmeansTM import ArabicKmeansTM

# Initialize and load dataset
dataset = TMDataset()
dataset.fetch_dataset("Arabic_News")

# Print basic information
print("\nDataset Info:")
print(f"Number of documents: {len(dataset.dataframe)}")
print(f"Available topics: {dataset.dataframe['labels'].unique()}")

c:\Users\yahya\my_projects\stream_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\yahya\my_projects\stream_env\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     c:\Users\yahya\my_projects\stream_env\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


PyArabic not available. Installing basic preprocessing...


2024-12-15 17:56:23.599 | INFO     | stream_topic.utils.dataset:fetch_dataset:230 - Fetching dataset: Arabic_News
2024-12-15 17:56:23.600 | INFO     | stream_topic.utils.dataset:fetch_dataset:261 - Processing Arabic dataset from c:\users\yahya\my_projects\stream\stream_topic\stream_topic_data\arabic_datasets\Arabiya
2024-12-15 17:56:23.600 | DEBUG    | stream_topic.utils.dataset:_load_arabic_dataset:134 - Tried path (not found): c:\users\yahya\my_projects\stream\data\arabic_datasets\arabic_news\Arabiya
2024-12-15 17:56:23.601 | DEBUG    | stream_topic.utils.dataset:_load_arabic_dataset:134 - Tried path (not found): c:\users\yahya\my_projects\stream\stream_topic\stream_topic_data\arabic_datasets\Arabiya
2024-12-15 17:56:23.602 | DEBUG    | stream_topic.utils.dataset:_load_arabic_dataset:134 - Tried path (not found): data\arabic_datasets\arabic_news\Arabiya
2024-12-15 17:56:23.602 | DEBUG    | stream_topic.utils.dataset:_load_arabic_dataset:134 - Tried path (not found): c:\Users\yahya\my


Dataset Info:
Number of documents: 11990
Available topics: ['Culture' 'Finance' 'Medical' 'Politics' 'Sports' 'Tech']


## Preprocessing Arabic Text

In [4]:
# Set the language and preprocess data
dataset.language = "ar"
dataset.preprocess()

# Print sample results
# print("Sample of first 5 documents being preprocessed:")
# for text in dataset.texts[:5]:
#     print(f"Processing text: {text[:100]}...")
# print("\nUnique labels:", dataset.dataframe['labels'].unique())
# print("\nNumber of documents per label:")
# print(dataset.dataframe['labels'].value_counts())

Found 11990 documents to preprocess

Sample of first 3 documents:
Processing text: غيب الموت مساء السبت، الفنان المصري محمود الدسوقي،...
Processing text: فاز الفيلم الإيراني "الصورة الأخيرة لذكرى" بالجائز...
Processing text: عبرت الفنانة الكويتية غدير السبتي عن سعادتها الغام...
...(remaining documents omitted)...

Preprocessing in progress...
Preprocessing completed successfully!


In [5]:
# After loading dataset but before training
print("Sample of preprocessed text:")
print(dataset.dataframe['text'].iloc[0])
print("\nUnique labels:", dataset.dataframe['labels'].unique())
print("\nNumber of documents per label:")
print(dataset.dataframe['labels'].value_counts())

Sample of preprocessed text:
غيب الموت السبت الفنان المصري محمود الدسوقي زوج عزة مجاهد ابنة الفنانة المصرية فيفي عبده إثر إصابته بأزمة قلبية حادة وشيعت جنازة الفنان الراحل مسجد مصطفى محمود بالمهندسين عقب صلاة المغرب ودفن مقابر الأسرة يقام العزاء بمسجد الحادمية الشاذلية ونعت الفنانة فيفي عبده زوج ابنتها صفحتها انستغرام وكتبت لله وإنا إليه راجعون توفي رجل الأعمال والفنان محمود الدسوقي زوج ابنتي عزة مجاهد ووالد ابنتهما شهد الدسوقي فيفي عبده وابنتها ابنه فيفي عبده وزوجها يبلغ الدسوقي العمر عاما وعمل عدد الأعمال الفنية وظهر ممثلا زوجته عزة وحماته الفنانة فيفي عبده العرض المسرحي حارة العوالم وقال التمثيل حماته يعطي طاقة كبيرة المسرح وإنها أعطته ثقة كبيرة تجسيد دوره ظهر فيفي عبده أغنيتها المصورة حرة بحبش ضرة وجسد شخصية زوجها

Unique labels: ['Culture' 'Finance' 'Medical' 'Politics' 'Sports' 'Tech']

Number of documents per label:
labels
Culture     2000
Medical     2000
Sports      2000
Tech        2000
Finance     1999
Politics    1991
Name: count, dtype: int64


## Training the Topic Model


In [6]:
# Initialize and train model
model = ArabicKmeansTM(num_topics=20)
model.fit(dataset)

# Get and display topics
topics = model.get_topics()
print("\nExtracted Topics:")
for idx, topic in enumerate(topics):
    print(f"\nTopic {idx}:")
    print(", ".join(topic[:10]))  # Print top 10 words

2024-12-15 17:57:06.410 | INFO     | stream_topic.models.ArabicKmeansTM:fit:227 - Training ArabicKmeansTM
2024-12-15 17:57:06.410 | INFO     | stream_topic.models.ArabicKmeansTM:fit:240 - Using TF-IDF approach
c:\Users\yahya\my_projects\stream_env\lib\site-packages\sklearn\feature_extraction\text.py:394: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['أفعل', 'أفعله', 'انفك', 'برح', 'سيما'] not in stop_words.
  warnings.warn(
c:\Users\yahya\my_projects\stream_env\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp


Extracted Topics:

Topic 0:
الأهلي, الهلال, السعودي, مباراة, النصر, الفريق, النادي, المباراة, الموسم, الاتحاد

Topic 1:
سوريا, النظام, حلب, المعارضة, السوري, السورية, قوات, الأسد, المرصد, داعش

Topic 2:
النفط, برميل, للبرميل, أوبك, الخام, يوميا, أسعار, برنت, الإنتاج, مخزونات

Topic 3:
الفنانة, الفنان, المسلسل, مسلسل, محمد, الدراما, المخرج, الأغنية, أحمد, الفنانين

Topic 4:
مباراة, الدوري, مدريد, القدم, الفريق, الموسم, برشلونة, المدرب, لكرة, المباراة

Topic 5:
البنك, الدولار, المركزي, البنوك, أسعار, مليار, الفائدة, بنك, سعر, العملة

Topic 6:
الجسم, تناول, الدم, النوم, الأطعمة, الدهون, الوزن, القلب, الغذائية, الإصابة

Topic 7:
التطبيق, تطبيق, أندرويد, المستخدم, التطبيقات, متجر, لمستخدمي, تطبيقات, الصور, للمستخدمين

Topic 8:
شركة, دبي, مليار, المملكة, الشركة, المالية, الشركات, السوق, الإمارات, مجلس

Topic 9:
آبل, آيفون, شركة, هواتف, هاتف, الشركة, آيباد, الهاتف, الذكية, سامسونغ

Topic 10:
الرئيس, الأميركية, إيران, وزير, الاتحاد, مجلس, الحكومة, الأمن, محمد, الدول

Topic 11:
الفيروس, الصحة,

## Model Evaluation

In [7]:
from stream_topic.metrics import ISIM, INT, ISH, Expressivity, NPMI

# Calculate ISIM score
metric = ISIM()
isim_score = metric.score(topics)
print(f"\nISIM Score: {isim_score}")

# Calculate per-topic scores
topic_scores = metric.score_per_topic(topics)
print("\nPer-topic scores:")
for topic, score in topic_scores.items():
    print(f"{topic[:50]}...: {score:.4f}")

c:\Users\yahya\my_projects\stream_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



ISIM Score: 0.7935564905405045

Per-topic scores:
الأهلي, الهلال, السعودي, مباراة, النصر...: 0.7896
سوريا, النظام, حلب, المعارضة, السوري...: 0.7890
النفط, برميل, للبرميل, أوبك, الخام...: 0.8372
الفنانة, الفنان, المسلسل, مسلسل, محمد...: 0.8264
مباراة, الدوري, مدريد, القدم, الفريق...: 0.8076
البنك, الدولار, المركزي, البنوك, أسعار...: 0.8541
الجسم, تناول, الدم, النوم, الأطعمة...: 0.6438
التطبيق, تطبيق, أندرويد, المستخدم, التطبيقات...: 0.8408
شركة, دبي, مليار, المملكة, الشركة...: 0.8017
آبل, آيفون, شركة, هواتف, هاتف...: 0.7268
الرئيس, الأميركية, إيران, وزير, الاتحاد...: 0.8411
الفيروس, الصحة, زيكا, فيروس, إصابة...: 0.7369
الدراسة, الباحثون, الإصابة, دراسة, القلب...: 0.8545
الشوط, الدقيقة, الحارس, المباراة, تمريرة...: 0.7950
الحوثي, داعش, القوات, التحالف, ميليشيات...: 0.8284
الهاتف, سامسونغ, غالاكسي, هاتف, شركة...: 0.7491
مليار, الربع, بنسبة, بقيمة, سهم...: 0.7971
فيسبوك, المستخدمين, تويتر, الميزة, المستخدم...: 0.7692
الفيلم, فيلم, المهرجان, المخرج, السينما...: 0.8475
غوغل, شركة, الشركة, م

## Visualizing Topics

In [10]:
from stream_topic.visuals import visualize_topic_model

visualize_topic_model(
    model, 
    reduce_first=True, 
    port=8051,
)

c:\Users\yahya\my_projects\stream_env\lib\site-packages\dash\dash.py:2282: DeprecationWarning:

Dash.run_server is deprecated and will be removed in Dash 3.0



## Optimizing Model Parameters

In [17]:
# # Optional: Perform hyperparameter optimization
# model.optimize_and_fit(
#     dataset,
#     min_topics=2,
#     max_topics=20,
#     criterion="aic",
#     n_trials=20,
# )